In [ ]:
#STUDENT NAME: SHRUTHI
#STUDENT ID: 801218392
#HOMEWORK 5 QUESTION 2

In [1]:
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import torch
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [2]:
housing = pd.read_csv('Housing.csv')
housing

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished
...,...,...,...,...,...,...,...,...,...,...,...,...,...
540,1820000,3000,2,1,1,yes,no,yes,no,no,2,no,unfurnished
541,1767150,2400,3,1,1,no,no,no,no,no,0,no,semi-furnished
542,1750000,3620,2,1,1,yes,no,no,no,no,0,no,unfurnished
543,1750000,2910,3,1,1,no,no,no,no,no,0,no,furnished


In [3]:
#Choosing the values of the columns as per the question
num_vars = ['price', 'area', 'bedrooms', 'bathrooms', 'stories', 'parking']
df_new = housing[num_vars]
df_new

,price,area,bedrooms,bathrooms,stories,parking
0,13300000,7420,4,2,3,2
1,12250000,8960,4,4,4,3
2,12250000,9960,3,2,2,2
3,12215000,7500,4,2,2,3
4,11410000,7420,4,1,2,2
...,...,...,...,...,...,...
540,1820000,3000,2,1,1,2
541,1767150,2400,3,1,1,0
542,1750000,3620,2,1,1,0
543,1750000,2910,3,1,1,0


In [4]:
df_new.shape

(545, 6)

In [5]:
#Scaling the data
scaler = MinMaxScaler()
df_new[num_vars] = scaler.fit_transform(df_new[num_vars])
df_new.head()

C:\Users\Shruthi\AppData\Local\Temp\ipykernel_14832\2943118898.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new[num_vars] = scaler.fit_transform(df_new[num_vars])


,price,area,bedrooms,bathrooms,stories,parking
0,1.000000,0.396564,0.6,0.333333,0.666667,0.666667
1,0.909091,0.502405,0.6,1.000000,1.000000,1.000000
2,0.909091,0.571134,0.4,0.333333,0.333333,0.666667
3,0.906061,0.402062,0.6,0.333333,0.333333,1.000000
4,0.836364,0.396564,0.6,0.000000,0.333333,0.666667


In [6]:
#Splitting the dataset into parameters and labels
X = df_new.iloc[:, 1:6].values
Y = df_new.iloc[:, 0].values

In [7]:
#Converting into tensors
X = torch.tensor(X)
Y = torch.tensor(Y)

In [8]:
#Defining the model as per the question
def model(X, W1, W2, W3, W4, W5, B):
    return W5*X[:,4] + W4*X[:,3] + W3*X[:,2] + W2*X[:,1] + W1*X[:,0] + B

In [9]:
#Defining the MSE
def loss_fn(Y_p, Y):
    squared_diffs = (Y_p - Y)**2
    return squared_diffs.mean()

In [10]:
#Defining the weights and biases as arrays of ones and zeros respectively 
W1 = torch.ones(())
W2 = torch.ones(())
W3 = torch.ones(())
W4 = torch.ones(())
W5 = torch.ones(())
B = torch.zeros(())

In [11]:
#Splitting the test and train ratio as 20-80 
n_samples = X.shape[0]
n_val = int(0.2 * n_samples)
#Shuffling the indices of the original dataset so that the model can be trained more effectively
shuffled_indices = torch.randperm(n_samples)

train_indices = shuffled_indices[:-n_val]
val_indices = shuffled_indices[-n_val:]

In [12]:
#Splitting the data into train and test and into parameters and target/labels
train_X = X[train_indices]
train_Y = Y[train_indices]

val_X = X[val_indices]
val_Y = Y[val_indices]

In [13]:
train_X.size()

torch.Size([436, 5])

In [14]:
#Defining the training loop using the model defined above with the validation and the training and printing selected epochs
def training_loop(n_epochs, optimizer, params, train_X, val_X, train_Y, val_Y):
    
    for epoch in range(1, n_epochs + 1):
        train_Y_p = model(train_X, *params) 
        train_loss = loss_fn(train_Y_p, train_Y)
                             
        val_Y_p = model(val_X, *params) 
        val_loss = loss_fn(val_Y_p, val_Y)
        
        optimizer.zero_grad()
        train_loss.backward() 
        optimizer.step()

        if epoch % 500 == 0:
            print(f"Epoch {epoch}, Training loss {train_loss.item():.4f},"
                  f" Validation loss {val_loss.item():.4f}")
            
    return params

In [15]:
#Initializing the parameters (weights and the biases) as ones and zeros respectively and running the training for 5000 epochs,  
#    using the SGD optimizer and with a learning rate of 0.1
params = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad=True)
learning_rate = 1e-1
optimizer = optim.SGD([params], lr=learning_rate)

training_loop(
    n_epochs = 5000, 
    optimizer = optimizer,
    params = params,
    train_X = train_X,
    val_X = val_X,
    train_Y = train_Y,
    val_Y = val_Y)

Epoch 500, Training loss 0.0121, Validation loss 0.0108
Epoch 1000, Training loss 0.0119, Validation loss 0.0103
Epoch 1500, Training loss 0.0118, Validation loss 0.0102
Epoch 2000, Training loss 0.0118, Validation loss 0.0102
Epoch 2500, Training loss 0.0118, Validation loss 0.0102
Epoch 3000, Training loss 0.0118, Validation loss 0.0102
Epoch 3500, Training loss 0.0118, Validation loss 0.0102
Epoch 4000, Training loss 0.0118, Validation loss 0.0102
Epoch 4500, Training loss 0.0118, Validation loss 0.0102
Epoch 5000, Training loss 0.0118, Validation loss 0.0102


tensor([0.3973, 0.0948, 0.3000, 0.1390, 0.1115, 0.0370], requires_grad=True)

In [16]:
#Initializing the parameters (weights and the biases) as ones and zeros respectively and running the training for 5000 epochs,  
#    using the SGD optimizer and with a learning rate of 0.01
params = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad=True)
learning_rate = 1e-2
optimizer = optim.SGD([params], lr=learning_rate)

training_loop(
    n_epochs = 5000, 
    optimizer = optimizer,
    params = params,
    train_X = train_X,
    val_X = val_X,
    train_Y = train_Y,
    val_Y = val_Y)

Epoch 500, Training loss 0.0499, Validation loss 0.0540
Epoch 1000, Training loss 0.0227, Validation loss 0.0239
Epoch 1500, Training loss 0.0161, Validation loss 0.0162
Epoch 2000, Training loss 0.0141, Validation loss 0.0137
Epoch 2500, Training loss 0.0133, Validation loss 0.0125
Epoch 3000, Training loss 0.0128, Validation loss 0.0118
Epoch 3500, Training loss 0.0125, Validation loss 0.0114
Epoch 4000, Training loss 0.0123, Validation loss 0.0111
Epoch 4500, Training loss 0.0122, Validation loss 0.0109
Epoch 5000, Training loss 0.0121, Validation loss 0.0108


tensor([ 0.4228,  0.2097,  0.2928,  0.1121,  0.0977, -0.0038],
       requires_grad=True)

In [17]:
#Initializing the parameters (weights and the biases) as ones and zeros respectively and running the training for 5000 epochs,  
#    using the SGD optimizer and with a learning rate of 0.001
params = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad=True)
learning_rate = 1e-3
optimizer = optim.SGD([params], lr=learning_rate)

training_loop(
    n_epochs = 5000, 
    optimizer = optimizer,
    params = params,
    train_X = train_X,
    val_X = val_X,
    train_Y = train_Y,
    val_Y = val_Y)

Epoch 500, Training loss 0.2201, Validation loss 0.2309
Epoch 1000, Training loss 0.1367, Validation loss 0.1475
Epoch 1500, Training loss 0.1159, Validation loss 0.1256
Epoch 2000, Training loss 0.1014, Validation loss 0.1100
Epoch 2500, Training loss 0.0892, Validation loss 0.0967
Epoch 3000, Training loss 0.0787, Validation loss 0.0854
Epoch 3500, Training loss 0.0697, Validation loss 0.0756
Epoch 4000, Training loss 0.0621, Validation loss 0.0673
Epoch 4500, Training loss 0.0555, Validation loss 0.0601
Epoch 5000, Training loss 0.0498, Validation loss 0.0539


tensor([ 0.6761,  0.5728,  0.6515,  0.3471,  0.3904, -0.3960],
       requires_grad=True)

In [18]:
#Initializing the parameters (weights and the biases) as ones and zeros respectively and running the training for 5000 epochs,  
#    using the SGD optimizer and with a learning rate of 0.0001
params = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad=True)
learning_rate = 1e-4
optimizer = optim.SGD([params], lr=learning_rate)

training_loop(
    n_epochs = 5000, 
    optimizer = optimizer,
    params = params,
    train_X = train_X,
    val_X = val_X,
    train_Y = train_Y,
    val_Y = val_Y)

Epoch 500, Training loss 0.9742, Validation loss 0.9759
Epoch 1000, Training loss 0.7796, Validation loss 0.7837
Epoch 1500, Training loss 0.6311, Validation loss 0.6370
Epoch 2000, Training loss 0.5176, Validation loss 0.5250
Epoch 2500, Training loss 0.4308, Validation loss 0.4392
Epoch 3000, Training loss 0.3643, Validation loss 0.3735
Epoch 3500, Training loss 0.3133, Validation loss 0.3231
Epoch 4000, Training loss 0.2739, Validation loss 0.2842
Epoch 4500, Training loss 0.2435, Validation loss 0.2541
Epoch 5000, Training loss 0.2199, Validation loss 0.2308


tensor([ 0.8425,  0.7597,  0.9053,  0.7710,  0.8044, -0.5074],
       requires_grad=True)